In [ ]:
# SUDOKU PROJECT

import tkinter as tk

window = tk.Tk()
window.title("Sudoku")
icon = PhotoImage(file= 'icon.png')
window.iconphoto(True, icon)

window.mainloop()

NameError: name 'PhotoImage' is not defined